In [1]:
from line_solver import *
import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
#Nested Fork-Join Network
#Fork-join structures within fork-join structures
model = Network('model')

In [3]:
# Create nested fork-join structure
source = Source(model, 'Source')

outer_fork = Fork(model, 'OuterFork')
inner_fork1 = Fork(model, 'InnerFork1')

inner_fork2 = Fork(model, 'InnerFork2')#Queues in nested structure
queue1 = Queue(model, 'Queue1', SchedStrategy.PS)

queue2 = Queue(model, 'Queue2', SchedStrategy.PS)
queue3 = Queue(model, 'Queue3', SchedStrategy.PS)

queue4 = Queue(model, 'Queue4', SchedStrategy.PS)
inner_join1 = Join(model, 'InnerJoin1', inner_fork1)

inner_join2 = Join(model, 'InnerJoin2', inner_fork2)
outer_join = Join(model, 'OuterJoin', outer_fork)

sink = Sink(model, 'Sink')
jobclass = OpenClass(model, 'Class1')

In [4]:
#Service configurations
source.setArrival(jobclass, Exp(0.3))
queue1.setService(jobclass, Exp(1.5))
queue2.setService(jobclass, Exp(1.2))
queue3.setService(jobclass, Exp(1.8))
queue4.setService(jobclass, Exp(1.0))

In [5]:
# Nested fork-join routing
P = model.initRoutingMatrix()#Outer level
P.set(jobclass, jobclass, source, outer_fork, 1.0)
P.set(jobclass, jobclass, outer_fork, inner_fork1, 1.0)
P.set(jobclass, jobclass, outer_fork, inner_fork2, 1.0)#Inner level 1
P.set(jobclass, jobclass, inner_fork1, queue1, 1.0)
P.set(jobclass, jobclass, inner_fork1, queue2, 1.0)
P.set(jobclass, jobclass, queue1, inner_join1, 1.0)
P.set(jobclass, jobclass, queue2, inner_join1, 1.0)#Inner level 2
P.set(jobclass, jobclass, inner_fork2, queue3, 1.0)
P.set(jobclass, jobclass, inner_fork2, queue4, 1.0)
P.set(jobclass, jobclass, queue3, inner_join2, 1.0)
P.set(jobclass, jobclass, queue4, inner_join2, 1.0)#Back to outer level
P.set(jobclass, jobclass, inner_join1, outer_join, 1.0)
P.set(jobclass, jobclass, inner_join2, outer_join, 1.0)
P.set(jobclass, jobclass, outer_join, sink, 1.0)
model.link(P)

In [6]:
# Solve nested system
solver = SolverJMT(model, 'samples', 20000)
avgTable = solver.getAvgTable()

JMT Model: /tmp/workspace/jsim/16077761416864889519/jmodel.jsim
      Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0      Source   Class1  0.0000  0.0000  0.0000  0.0000  0.0000  0.2979
1      Queue1   Class1  0.2435  0.1946  0.8339  0.2085  0.2979  0.2979
2      Queue2   Class1  0.3341  0.2504  1.1050  0.2763  0.2979  0.2977
3      Queue3   Class1  0.1945  0.1632  0.6714  0.1678  0.2979  0.2979
4      Queue4   Class1  0.4120  0.2893  1.3683  0.3421  0.2979  0.2985
5  InnerJoin1   Class1  0.3021  0.0000  0.4990  0.2495  0.5978  0.2977
6  InnerJoin2   Class1  0.3390  0.0000  0.5742  0.2871  0.5888  0.2985
7   OuterJoin   Class1  0.3870  0.0000  0.6496  0.6496  0.5889  0.2984
